In [16]:
import mediapipe as mp
import numpy as np
import cv2
import math
import pandas as pd

In [17]:
facemesh =  mp.solutions.face_mesh
face = facemesh.FaceMesh(
    static_image_mode=False, 
    min_tracking_confidence=0.5, 
    min_detection_confidence=0.5
    )
draw = mp.solutions.drawing_utils

In [18]:
def EARandMAR(right_eye, left_eye, mouth):
    right_eye = (eucledianDistance(right_eye[2], right_eye[5]) + eucledianDistance(right_eye[4], right_eye[3]))/(2*eucledianDistance(right_eye[0], right_eye[1]))
    left_eye = (eucledianDistance(left_eye[4], left_eye[3]) + eucledianDistance(left_eye[5], left_eye[2]))/(2*eucledianDistance(left_eye[1], left_eye[0]))
    mouth = (eucledianDistance(mouth[0], mouth[2]) + eucledianDistance(mouth[3], mouth[5]))/(2*eucledianDistance(mouth[1], mouth[4]))
    return right_eye, left_eye, mouth

def eucledianDistance(point1, point2):
    return math.sqrt((point2.x - point1.x)**2 + (point2.y - point1.y)**2)

In [19]:
cap =  cv2.VideoCapture(0)
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [
    #upper right mouth
    # 82, 
    38, 
    # 72, 37,

    # 81, 41, 73, 39,
    # 80, 42, 74, 40,
    # 183,184, 185, 191,
    
    # 61, 
    76, 
    # 62, 78,

    #upper middle mouth
    # 14,  13, 0, 11,

    #upper left mouth
    # 312, 
    268, 
    # 302, 267,
    
    # 311, 271, 303, 269,
    # 310, 272, 304, 270,
    # 415, 407, 408, 409,
    
    # 291, 
    292,
    #  308, 306,
    
    # 324,325, 307, 375,

    #botom right mouth
    # 146, 77,
    # 88, 89,90, 91,
    # 178, 179, 180, 181,
    
    # 87, 
    86, 
    # 85, 84,

    #botom middle mouth
    # 12, 15, 16, 17,

    #botom left mouth
    # 318, 319, 320, 321,
    # 402, 403, 404, 405,
    
    # 317,
    316, 
    # 315, 314
    ]

face_data = {
    "right_eyes" : [],
    "left_eyes" : [],
    'mouth' : []
    }

while True:
    _, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgH, imgW, _ = frame.shape
    # print(imgH, imgW)
    rgb =  cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    op = face.process(rgb)
    if op.multi_face_landmarks:
        for pt in op.multi_face_landmarks:
            # draw.draw_landmarks(frame, i)
            temp = [[],[],[]]
            left = 1
            right = 1
            mouth = 1
            for no,point in enumerate(pt.landmark):
                if no in chosen_left_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, color, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{left}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[0].append(point)
                    left += 1
                if no in chosen_right_eye_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (color, 0, 0), -1)
                    cv2.putText(
                        frame, text= str(f'p{right}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[1].append(point)
                    right += 1
                if no in chosen_mouth_idxs:
                    # cv2.circle(frame, (int(point.x * imgW), int(point.y * imgH)), 2, (0, 0, color), -1)
                    cv2.putText(
                        frame, text= str(f'p{mouth}'), org=(int(point.x * imgW), int(point.y * imgH)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0,255,0),
                        thickness=1, lineType=cv2.LINE_AA
                    )
                    temp[2].append(point)
                    mouth += 1
            right_eye, left_eye, mouth = EARandMAR(temp[0], temp[1], temp[2])
            face_data['right_eyes'].append(right_eye)
            face_data['left_eyes'].append(left_eye)
            face_data['mouth'].append(mouth)
            if right_eye < 0.3 and left_eye < 0.3:
                cv2.putText(
                        frame, text= str(f'Mata Tertutup'), org=(20, 20),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,0,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
            else:
                cv2.putText(
                        frame, text= str(f'Mata Terbuka '), org=(20, 20),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
                
            if mouth > 0.5:
                cv2.putText(
                        frame, text= str(f'Driver Menguap'), org=(20, 40),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(255,0,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
            else:
                cv2.putText(
                        frame, text= str(f'Driver Tidak Menguap'), org=(20, 40),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,255,0),
                        thickness=2, lineType=cv2.LINE_AA
                    )
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("windows", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    # break

# df = pd.DataFrame(face_data)
print(face_data)
cap.release()
cv2.destroyAllWindows()

{'right_eyes': [0.25597333531599986, 0.22410098066629353, 0.20698043600842003, 0.23722620327638988, 0.21397092732835207, 0.27015046593936337, 0.2542121179343993, 0.25375481233743113, 0.26004301096465815, 0.2590207162603013, 0.24334996871561354, 0.23953163728954016, 0.23894219314900011, 0.24576215467256182, 0.2393765381423792, 0.25279119914630577, 0.2612224520544692, 0.2585906469923807, 0.25879232993104356, 0.2647216285397674, 0.26555630861734125, 0.25261373797025205, 0.253859590274916, 0.2606014849333646, 0.2652317766876865, 0.2767531002403402, 0.2919967496470311, 0.2966875172294935, 0.3032777176888179, 0.3061220344001232, 0.31802541501076154, 0.31961608703144734, 0.3194739095746136, 0.3240247840533626, 0.29609366632134926, 0.28933927822773686, 0.2984778284139856, 0.30056913500753846, 0.28173786060019723, 0.2776359052848169, 0.28237670400520526, 0.2744032018186146, 0.2617815359898518, 0.2628513618517573, 0.2549345608675657, 0.2963225841523277, 0.28190865297787226, 0.2880671470168134, 0

In [20]:
cap.release()
cv2.destroyAllWindows()